# Ballpark Factor

Load the necessary packages

In [46]:
from lxml.html import parse
import pandas as pd

the team names and numbers

In [47]:
teams =["BC", "GOC", "GRC", "HU", "INWU", "MAR", "MVNU", "SAU", "SFIN", "TAYL"]
t_nums = [1629, 1678, 1679, 1688, 1694, 1717, 1736, 1780, 1805, 1784]

Scrape the webpages

In [66]:
urls = ['http://www.dakstats.com/WebSync/Pages/Team/TeamSchedule.aspx?association=10&sg=MBA&sea=NAIMBA_2019&team=' +
        str(num) for num in t_nums]
#Parse the html page into a tree (specifically an ElementTree object)
pages = [parse(url) for url in urls]
#Get the tables from the webpage
tables = [page.xpath("//table") for page in pages]

The following code shows us that the header is at 33 and the rows are at 35.

In [71]:
for i in range(len(tables[2])):
  rows = tables[2][i].findall("tr")
  data = [[c.text for c in row.getchildren()] for row in rows]
  print(i, data)

0 [['\r\n                    ']]
1 [['\r\n            ', '\xa0']]
2 [['\r\n                        ', '\r\n                    ', '\r\n                        '], ['\r\n                        '], ['\r\n                        '], [None]]
3 [['\r\n                                    ', '\r\n                                    ', '\r\n                                    ']]
4 [['\r\n                                    ', '\r\n                                    ', '\r\n                                    ']]
5 [[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]]
6 [[None]]
7 [[None]]
8 [[None]]
9 [[None]]
10 [[None]]
11 [[None]]
12 [[None]]
13 [[None]]
14 [[None]]
15 [[None]]
16 [[None]]
17 [[None]]
18 [['\r\n                ', '\r\n                ', '\r\n                ', '\r\n                ', '\r\n                ', '\r\n                ', '\r\n                '], [None, '\r\n                ',

data[35]

In [73]:
rows = tables[2][35].findall("tr")
data = [[c for c in row.getchildren()] for row in rows]

In [74]:
frow = data[1]

In [56]:
c_rows = [tab[33].findall("tr") for tab in tables]
cols = [[[c.text for c in row.getchildren()] for row in team_rows] for team_rows in c_rows]
d_rows = [tab[35].findall("tr") for tab in tables]
cdata = [[[c.text for c in list(row.getchildren())[:5]] for row in team_rows if len(row) > 1] for team_rows in d_rows]
dfc = [pd.DataFrame(columns = cols[i], data = cdata[i]) for i in range(len(cols))]
dfc[3]

,Date,Opponent,H/A,Time,W/L
0,2/8/2019,None,A,None,W
1,2/8/2019,None,A,None,W
2,2/9/2019,None,N,None,W
3,2/22/2019,None,N,None,L
4,2/22/2019,None,N,None,L
5,3/1/2019,None,A,None,L
6,3/1/2019,None,A,None,W
7,3/2/2019,None,A,None,L
8,3/2/2019,None,A,None,L
9,3/8/2019,None,A,None,W


Print the contents of our raw data to find the table.

It appears to be line 44 and the odd lines between 45-133.

In [23]:
for i in range(len(tables[2])):
  print(i, [tab for tab in list(tables[2][i].iterchildren())])

0 [<Element tr at 0x7ffbdb04c778>]
1 [<Element tr at 0x7ffbdb04c7c8>]
2 [<Element tr at 0x7ffbdb04c818>, <Element tr at 0x7ffbdb04c8b8>, <Element tr at 0x7ffbdb04c958>, <Element tr at 0x7ffbdb04c9a8>]
3 [<Element tr at 0x7ffbdb04c868>]
4 [<Element tr at 0x7ffbdb04c908>]
5 [<Element tr at 0x7ffbdb04c9f8>]
6 [<Element tr at 0x7ffbdb04ca48>]
7 [<Element tr at 0x7ffbdb04ca98>]
8 [<Element tr at 0x7ffbdb04cae8>]
9 [<Element tr at 0x7ffbdb04cb38>]
10 [<Element tr at 0x7ffbdb04cb88>]
11 [<Element tr at 0x7ffbdb04cbd8>]
12 [<Element tr at 0x7ffbdb04cc28>]
13 [<Element tr at 0x7ffbdb04cc78>]
14 [<Element tr at 0x7ffbdb04ccc8>]
15 [<Element tr at 0x7ffbdb04cd18>]
16 [<Element tr at 0x7ffbdb04cd68>]
17 [<Element tr at 0x7ffbdb04cdb8>]
18 [<Element tr at 0x7ffbdb04ce08>, <Element tr at 0x7ffbdb04ce58>]
19 [<Element tr at 0x7ffbdb04cea8>, <Element tr at 0x7ffbdb050098>]
20 [<Element tr at 0x7ffbdb04cef8>, <Element tr at 0x7ffbdb04d408>, <Element tr at 0x7ffbdb04d458>, <Element tr at 0x7ffbdb050048>

In [20]:
a = list(tables[2][21].iterchildren())
print(a.text_content())

AttributeError: ignored

In [18]:
for i in range(len(tr_elements[2])):
    print(i, len(tr_elements[2][i]), [t.text_content() for t in list(tr_elements[2][i].iterchildren())])

0 1 ['\r\n                    \r\n                ']
1 2 ["\r\n            \r\n                \r\n                    \r\n                        \r\n                    \r\n                    \r\n                    \r\n\t\t\t\t    \r\n                        \r\n\t\r\n\t\t\r\n                                    Browse by School\r\n                                \r\n\t\t\r\n                                    Member Login\r\n                                \r\n\t\t\r\n                                    FAQ's\r\n                                \r\n\t\r\n\r\n\r\n                    \r\n                \r\n                \r\n\t\t\t\t    \r\n                        \r\n                            \r\n                                \r\n                                    Scoreboards\r\n                                \r\n                                \r\n                                    Webcasts\r\n                                \r\n                                \r\n         

In [ ]:
lines = [44] + list(range(47,134,2))
C_elements = [[
               T_els[i]
               for i in lines #pick only the lines we want
               ]
              for T_els in tr_elements #for each team
              ]

Notice we still have a problem with junk in the last entry of every row except the header.  We'll remove that junk with a `[:5]` at the end of rows in `C_Data` below.

In [ ]:
for T in C_elements[2]:
  print(len(T), [t.text_content() for t in list(T.iterchildren())])

In [ ]:
from datetime import datetime

def nice_data(d):
  #If we can turn it into an integer, then do that
  try:
    d=int(d)
  except:
  #If it's not an integer, it might be a float
    try:
      d = float(d)
    except:
      #If it's not a float, it might be a date
      try:
        d = datetime.strptime(d, '%m/%d/%Y')
      except:
        pass
  return(d)


In [ ]:
C_Data = [[[
    nice_data(d.text_content()) #get the text content and turn it into numbers if possible
    for d in list(row.iterchildren())[:5] #for each of the first five elements in the row
    ]
    for row in C_els #for each row in the list of rows
    ]
    for C_els in C_elements #for each list of rows corresponding to each team
]
dfc = [pd.DataFrame(columns = D[0], data = D[1:]) for D in C_Data]
dfc[2]

,Date,Opponent,H/A,Time,W/L
0,2019-02-27,Lourdes (Ohio),N,3-4,L
1,2019-02-27,Lourdes (Ohio),N,4-8,L
2,2019-03-02,Cornerstone (Mich.),N,3-4,L
3,2019-03-02,Trinity Baptist,N,5-1,W
4,2019-03-04,Michigan-Dearborn,N,13-1,W
5,2019-03-05,Rochester (Mich.),N,24-4,W
6,2019-03-06,Robert Morris (Ill.),N,10-9,W
7,2019-03-08,Bethel (Ind.) *,N,13-6,W
8,2019-03-09,Bethel (Ind.) *,N,14-2,W
9,2019-03-09,Bethel (Ind.) *,N,3-1,W


In [ ]:
dfc2 = [df[df['Opponent'].str.contains("*", regex = False)]
        for df in dfc]

ValueError: ignored

In [ ]:
dfc[0]

,Date,Opponent,H/A,Time,W/L
0,2018-09-22 00:00:00,Trinity International (Ill.),H,5-0,W
1,2018-09-22 00:00:00,Trinity International (Ill.),H,6-3,W
2,2018-09-29 00:00:00,Michigan-Dearborn,A,9-12,L
3,2018-09-29 00:00:00,Michigan-Dearborn,A,3-13,L
4,2019-02-22 00:00:00,Olivet Nazarene (Ill.),N,0-2,L
5,2019-02-22 00:00:00,Olivet Nazarene (Ill.),N,2-7,L
6,2019-02-25 00:00:00,Union (Ky.),A,4-7,L
7,2019-02-25 00:00:00,Union (Ky.),A,6-7,L
8,2019-03-02 00:00:00,The Apprentice School,N,3-4,L
9,2019-03-02 00:00:00,The Apprentice School,N,6-3,W


In [ ]:
df[df['Opponent'].str.contains("*", regex = False)]

,Date,Opponent,H/A,Time,W/L
7,2019-03-08,Bethel (Ind.) *,N,13-6,W
8,2019-03-09,Bethel (Ind.) *,N,14-2,W
9,2019-03-09,Bethel (Ind.) *,N,3-1,W
10,2019-03-14,Taylor (Ind.) *,A,5-15,L
11,2019-03-16,Taylor (Ind.) *,A,2-10,L
12,2019-03-16,Taylor (Ind.) *,A,3-9,L
13,2019-03-23,Spring Arbor (Mich.) *,A,2-4,L
14,2019-03-25,Spring Arbor (Mich.) *,A,1-11,L
15,2019-03-25,Spring Arbor (Mich.) *,A,6-7,L
18,2019-03-28,Huntington (Ind.) *,H,12-2,W
